In [161]:
if 'feature_engine' in sys.modules:  
    del sys.modules["feature_engine"]
from feature_engine import Items, Interactions

In [162]:
items = Items()

In [163]:
interactions = Interactions()

In [160]:
# interactions.train[interactions.train.visitorid == 689]
# interactions.test[interactions.test.visitorid == 689]
# # items.items[items.items['itemid']==105824]
# # 105824

,visitorid,itemid,rating
109930,689,414593,1


In [166]:
items.items = items.cleanup_items(items.items, interactions.train)
interactions.train = items.cleanup_items(interactions.train, items.items)
interactions.test = items.cleanup_items(interactions.test, items.items)

interactions.test = interactions.processing_testset(interactions.test, interactions.train)


In [168]:
items.run_unit_tests(items.items, interactions.train)
interactions.run_unit_tests(interactions.test, interactions.train)

In [174]:
n_users=len(np.unique(interactions.train['visitorid']))
n_items=len(np.unique(items.items))

In [175]:
from lightfm import LightFM
from lightfm.evaluation import auc_score, precision_at_k, recall_at_k
from scipy.sparse import coo_matrix
from sklearn import preprocessing

In [178]:
id_cols=['visitorid','itemid']
trans_cat_train=dict()
trans_cat_test=dict()

cate_enc_dict = dict()
for k in id_cols:
    cate_enc_dict[k] = preprocessing.LabelEncoder()
    trans_cat_train[k]=cate_enc_dict[k].fit_transform(interactions.train[k].values)
    trans_cat_test[k]=cate_enc_dict[k].transform(interactions.test[k].values)

In [181]:
rate_matrix = dict()
rate_matrix['train'] = coo_matrix((interactions.train['rating'], (trans_cat_train['visitorid'], trans_cat_train['itemid'])), shape=(n_users,n_items))
rate_matrix['test'] = coo_matrix((interactions.test['rating'], (trans_cat_test['visitorid'], trans_cat_test['itemid'])), shape=(n_users,n_items))


In [182]:
model = LightFM(no_components=10, loss='warp')
model.fit(rate_matrix['train'], epochs=100, num_threads=8)

In [183]:
def predict_recom(user_id_orig, recom_num, model):
    result_dict = dict()
    result_array = list()
    user_id = cate_enc_dict['visitorid'].transform([user_id_orig])[0]
    #Known Positives
    known_positives = np.unique(np.concatenate((events_train['itemid'].values[np.where(trans_cat_train['visitorid']==user_id)]
                    , events_test['itemid'].values[np.where(trans_cat_test['visitorid']==user_id)]), axis=0))
    
    #Recommended
    scores = np.unique(model.predict(user_ids = [user_id], item_ids = trans_cat_test['itemid']))
    scores_list = dict()
    top_items = trans_cat_test['itemid'][np.argsort(-scores)]
    print("TOP",len(scores))
    print(len(trans_cat_test['itemid']))

    # printing out the result
    print("User %s" % cate_enc_dict['visitorid'].inverse_transform([user_id])[0])
    print("     Known positives:")

    for x in known_positives[:recom_num]:
        print("                  %s" % x)


    print("     Recommended:")

    for x in top_items[:recom_num]:
        print("                  %s" % cate_enc_dict['itemid'].inverse_transform([x])[0])
        result_array.append(x)
    results_dict[user_id_orig] = result_array
    print("")
    return results_dict
    

In [184]:
for usr in np.array(events_test['visitorid'])[:2]:
    top_rec = predict_recom(usr, 5, model)

NameError: name 'events_test' is not defined

In [187]:
import feature_extractor
# sys.path.insert(1, '/path/to/application/app/folder')

In [188]:
import sys
sys.path

['/Users/awaiskaleem/Documents/Study/git/E-Commerce-Recommendation-Engine/etl',
 '/Users/awaiskaleem/anaconda3/lib/python37.zip',
 '/Users/awaiskaleem/anaconda3/lib/python3.7',
 '/Users/awaiskaleem/anaconda3/lib/python3.7/lib-dynload',
 '',
 '/Users/awaiskaleem/anaconda3/lib/python3.7/site-packages',
 '/Users/awaiskaleem/anaconda3/lib/python3.7/site-packages/aeosa',
 '/Users/awaiskaleem/Documents/Study/git/titanic/packages/regression_model',
 '/Users/awaiskaleem/anaconda3/lib/python3.7/site-packages/IPython/extensions',
 '/Users/awaiskaleem/.ipython']